In [1]:
import pandas as pd
import numpy as np
import random
import json
import matplotlib.pyplot as plt
from matplotlib_venn import venn3,venn2,venn3_circles
from datasets import load_dataset
from utils import *
import xml.etree.ElementTree as ET
pd.set_option('display.max_colwidth', None)

In [2]:
df_mm_all = pd.read_csv('../../inference/results/qwen/results-all-qwen-mm.csv')
df_mm_cot = pd.read_csv('../../inference/results/qwen/qwen-mm-cot.csv')
df_mm_sub = pd.read_csv('../../inference/results/qwen/qwen-mm-sub.csv')
df_mm_pot = pd.read_csv('../../inference/results/qwen/qwen-mm-pot.csv')
# df_gsm_test_all = pd.read_csv('../../inference/results/qwen/results-all-qwen-test.csv')
# df_gsm_train_all = pd.read_csv('../../inference/results/qwen/results-all-qwen-train.csv')
# df_svamp_all = pd.read_csv('../../inference/results/qwen/results-all-qwen-svamp.csv')
# df_asdiv_all = pd.read_csv('../../inference/results/qwen/results-all-qwen-asdiv.csv')
# df_gsm_train_cot = pd.read_csv('../../inference/results/qwen/qwen-gsm-train-cot.csv')
# df_gsm_train_sub = pd.read_csv('../../inference/results/qwen/qwen-gsm-train-sub.csv')
# df_gsm_train_pot = pd.read_csv('../../inference/results/qwen/qwen-gsm-train-pot.csv')

In [3]:
df_mm_all['question'] = df_mm_all['question'].apply(lambda x: x[:-1])
# df_gsm_test_all['question'] = df_gsm_test_all['question'].apply(lambda x: x[:-1])
# df_gsm_train_all['question'] = df_gsm_train_all['question'].apply(lambda x: x[:-1])
# df_svamp_all['question'] = df_svamp_all['question'].apply(lambda x: x[:-1])
# df_asdiv_all['question'] = df_asdiv_all['question'].apply(lambda x: x[:-1])

In [4]:
with open('/cluster/project/sachan/liuron/Thesis/experiments/SVAMP.json','r') as json_file:
    dataset_svamp = json.load(json_file)
dataset_gsm = load_dataset('gsm8k','main')['test']
dataset_asdiv = ET.parse('/cluster/project/sachan/liuron/Thesis/experiments/ASDiv.xml')

In [5]:
def xml_to_dict(element):
    # If the element has no children, return its text content
    if len(element) == 0:
        return element.text

    # Otherwise, create a dictionary for the element
    result = {}
    for child in element:
        child_result = xml_to_dict(child)
        if child.tag not in result:
            result[child.tag] = child_result
        else:
            # If the tag already exists, make it a list of results
            if not isinstance(result[child.tag], list):
                result[child.tag] = [result[child.tag]]
            result[child.tag].append(child_result)
    
    # Include element attributes if they exist
    if element.attrib:
        result['@attributes'] = element.attrib

    return result

# Convert the root element to a dictionary
root = dataset_asdiv.getroot()
xml_dict = {root.tag: xml_to_dict(root)}
dataset_asdiv = xml_dict['Machine-Reading-Corpus-File']['ProblemSet']['Problem']

In [6]:
#Plot the venn diagram to visualize how each strategy performs.
def plot_venn_diagram(cot,sub,pot,dataset,model):
    cot = set([index for index, value in enumerate(cot) if value])
    sub = set([index for index, value in enumerate(sub) if value])
    pot = set([index for index, value in enumerate(pot) if value])

    only_cot = len(cot - sub - pot)
    only_sub = len(sub - cot - pot)
    only_pot = len(pot - sub - cot)

    cot_sub = len(cot & sub - pot)
    cot_pot = len(cot & pot - sub)
    pot_sub = len(pot & sub - cot)
    
    cot_sub_pot = len(cot & sub & pot)
    
    plt.figure(figsize=(8, 8))
    v=venn3(subsets=(only_cot,only_sub,cot_sub,only_pot,cot_pot,pot_sub,cot_sub_pot),set_labels=('CoT','L2M','PoT'))
    c=venn3_circles(subsets=(only_cot,only_sub,cot_sub,only_pot,cot_pot,pot_sub,cot_sub_pot),linewidth=0.2)

    plt.title(f"Venn Diagram of CoT, L2M, and PoT on {dataset}, {model}")
    plt.show()

# plot_venn_diagram(df_mm_all['cot-correct'],df_mm_all['sub-correct'],df_mm_all['pot-correct'],'MetaMathQA','Gemma')
# plot_venn_diagram(df_gsm_test_all['cot-correct'],df_gsm_test_all['sub-correct'],df_gsm_test_all['pot-correct'],'GSM-Train','Gemma')
# plot_venn_diagram(df_gsm_train_all['cot-correct'],df_gsm_train_all['sub-correct'],df_gsm_train_all['pot-correct'],'GSM-Test','Gemma')
# plot_venn_diagram(df_svamp_all['cot-correct'],df_svamp_all['sub-correct'],df_svamp_all['pot-correct'],'SVAMP','Gemma')
# plot_venn_diagram(df_asdiv_all['cot-correct'],df_asdiv_all['sub-correct'],df_asdiv_all['pot-correct'],'ASDiv','Gemma')

In [7]:
example_prompt_1_hint = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
Solve the following Math Word Problem using the given strategy.

Input:
Question: {}
Given strategy: {}

Response:
{}"""

example_prompt_1 = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
Solve the following Math Word Problem by firstly generating a strategy, then solving the problem according to the generated strategy.

Input:
Question: {}
<incorrect>None

Response:
<correct>{}: {}"""

example_prompt_2 = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
Solve the following Math Word Problem by firstly generating a strategy, then solving the problem according to the generated strategy.

Input:
Question: {}
<incorrect>{}

Response:
<correct>{}: {}"""

In [8]:
strategy_correct_map = {
    # 'Answer Only': df_mm_all['ao-correct'],
    'Chain of Thought': df_mm_all['cot-correct'],
    'Sub-questioning': df_mm_all['sub-correct'],
    'Program of Thought': df_mm_all['pot-correct'],
    'Unsolvable': ~df_mm_all['correct']
}

strategy_output_map = {
    # 'Answer Only': df_mm_ao,
    'Chain of Thought': df_mm_cot,
    'Sub-questioning': df_mm_sub,
    'Program of Thought': df_mm_pot
}

def create_text_2step(df_all):
    result1 = []
    result2 = []
    questions = df_all['question']

    for i in range(len(df_all)):
        for strategy1 in ['Chain of Thought','Sub-questioning','Program of Thought']:
            chain1 = strategy_output_map[strategy1]['output_answer'][i] + '<eos>' if strategy1 != 'Unsolvable' else '<eos>'
            if strategy_correct_map[strategy1][i]:
                text1 = example_prompt_1.format(questions[i],strategy1,chain1)
                result1.append(text1)
            else:
                # chain1 = chain1.replace('<eos>','')
                for strategy2 in ['Chain of Thought','Sub-questioning','Program of Thought']:
                    if strategy_correct_map[strategy2][i]:
                        chain2 = strategy_output_map[strategy2]['output_answer'][i] + '<eos>' if strategy2 != 'Unsolvable' else '<eos>'
                        text2 = example_prompt_2.format(questions[i],strategy1,strategy2,chain2)
                        # text2 = example_prompt_2.format(questions[i],strategy2,chain2)
                        result2.append(text2)
    # print(result2[0])
    return result1, result2

def create_text_hint(df_all):
    result1 = []
    questions = df_all['question']

    for i in range(len(df_all)):
        # if not (strategy_correct_map['Chain of Thought'][i] and strategy_correct_map['Sub-questioning'][i] and strategy_correct_map['Program of Thought'][i]):
        for strategy1 in ['Chain of Thought','Sub-questioning','Program of Thought']:
            chain1 = strategy_output_map[strategy1]['output_answer'][i] + '<eos>'
            if strategy_correct_map[strategy1][i]:
                # text1 = example_prompt_1.format(questions[i],strategy1,chain)
                text1 = example_prompt_1_hint.format(questions[i],strategy1,chain1)
                # text2 = example_prompt_2_contexted.format(questions[i],strategy1,chain1,strategy1,chain1)
                result1.append(text1)
    return result1

def split_data(texts,sample_ratio=0.8):
    random.shuffle(texts) 
    train_size = int(len(texts) * sample_ratio)
    sampled_questions = texts[:train_size]
    unsampled_questions = texts[train_size:]
    return sampled_questions, unsampled_questions
    

In [9]:

def prepare_data(df):
    texts1,texts2 = create_text_2step(df)
    print(len(texts1),len(texts2))
    train,val = split_data(texts1+texts2,sample_ratio=0.9)
    print(len(val),len(train))
    # for i in range(10):
    #     print(train[i])
    dict_train = {
        'text' : train
    }
    dict_val = {
        'text' : val
    }

    with open('data/train.json', "w") as json_file:
        json.dump(dict_train, json_file)
    with open('data/val.json', "w") as json_file:
        json.dump(dict_val, json_file)


In [10]:
prepare_data(df_mm_all)

292581 62388
35497 319472


In [11]:
example_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
Solve the following Math Word Problem by firstly generating a strategy, then solving the problem according to the generated strategy.

Input:
Question: {}
<incorrect>None

Response:
<correct>"""

example_prompt_cot = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
Solve the following Math Word Problem by firstly generating a strategy, then solving the problem according to the generated strategy.

Input:
Question: {}
<incorrect>Sub-questioning
<incorrect>Program of Thought

Response:
<correct>"""

example_prompt_sub = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
Solve the following Math Word Problem by firstly generating a strategy, then solving the problem according to the generated strategy.

Input:
Question: {}
<incorrect>Chain of Thought
<incorrect>Program of Thought

Response:
<correct>"""

example_prompt_pot = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
Solve the following Math Word Problem by firstly generating a strategy, then solving the problem according to the generated strategy.Input:
Question: {}
<incorrect>Chain of Thought
<incorrect>Sub-questioning

Response:
<correct>"""

In [12]:
def create_test_data(data,name):
    if name == 'gsm':
        questions = data['question']
        answers = [extract_pred(data['answer'][i]) for i in range(len(data['answer']))]
    if name == 'svamp':
        questions = [x['Body'] + x['Question'] for x in data]
        answers = [x['Answer'] for x in data]
    if name == 'asdiv':
        questions = [x['Body'] + '' + x['Question'] for x in data]
        answers = [x['Answer'].split(' ')[0] for x in data]
        filtered_questions = []
        filtered_answers = []
        for i in range(len(answers)):
            try:
                ans = float(answers[i])
                filtered_questions.append(questions[i])
                filtered_answers.append(answers[i])
            except:
                continue
        questions = filtered_questions
        answers = filtered_answers
    texts_cot = [example_prompt_cot.format(questions[i],'') for i in range(len(questions))]
    texts_sub = [example_prompt_sub.format(questions[i],'') for i in range(len(questions))]
    texts_pot = [example_prompt_pot.format(questions[i],'') for i in range(len(questions))]
    texts = [example_prompt.format(questions[i],'') for i in range(len(questions))]
    # for i in range(5):
    #     print(texts[i])
    #     print()
    dict_test_cot = {'text':texts_cot,
                'correct-answer':answers,
                }
    dict_test_sub = {'text':texts_sub,
                'correct-answer':answers,
                    }
    dict_test_pot = {'text':texts_pot,
                'correct-answer':answers,
                }
    dict_test = {'text':texts,
                'correct-answer':answers,
                }
    with open(f'../epoch2/data/test_{name}_cot.json', "w") as json_file:
        json.dump(dict_test_cot, json_file)
    with open(f'../epoch2/data/test_{name}_sub.json', "w") as json_file:
        json.dump(dict_test_sub, json_file)
    with open(f'../epoch2/data/test_{name}_pot.json', "w") as json_file:
        json.dump(dict_test_pot, json_file)
    with open(f'../epoch2/data/test_{name}.json', "w") as json_file:
        json.dump(dict_test, json_file)
        
create_test_data(dataset_gsm,'gsm')
create_test_data(dataset_svamp,'svamp')
create_test_data(dataset_asdiv,'asdiv')